#### Annette Donald
#### Take-Home #1
#### Due: Tuesday 10/11/22

#### Background: The computational project is designed with the goal of understanding the spatial distribution of compliance jobs in the United States. According to the U.S. Bureau of Labor Statistics, in 2021, there were a total of 334,340 compliance officers employed in the United States. Most of them are in charge of organizations’ internal compliance programs, under job titles such as “Occupational Health and Safety Specialist”, “Air Pollution Compliance Inspector”, and “Equal Employment Opportunity Investigator”. Despite the growing size of this job category, there is little understanding of its overall topology. How are these jobs distributed spatially as well as across industry? What range of mandates do compliance offices address? Which regulatory mandates generate the largest amount of compliance activity and which the least? 

In [4]:
import os 
import pandas as pd 
import geopandas as gpd
import seaborn as sns 
import folium
import random
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
from shapely.geometry import Point

#### Task 1: Integrate information from the “job_url_data” folder into one dictionary. The keys of the dictionary are individual urls scraped from the website, and the values are the earliest date that the corresponding urls were scraped. Save the results into a json file. How many unique job urls have been collected between May 17, 2022 and May 23, 2022? 

#### __21,260__ unique job URLs were collected between May 17, 2022 and May 23, 2022.

In [5]:
# Read CSV into DF
#job_url_data_df = pd.DataFrame()#pd.read_csv("/Users/annettedblackburn/Desktop/CompSoc/Take_Home_1/indeed_scraped_data/job_url_data/job_urls_for_parsehub_5172022_v1.csv")
file_root = "/Users/annettedblackburn/Desktop/CompSoc/Take_Home_1/indeed_scraped_data/job_url_data"
dfs = []
for file in os.listdir(file_root):
    if file == ".DS_Store": continue
    file_path = f"{file_root}/{file}"
    dfs += [pd.read_csv(file_path)]

job_url_data_df = pd.concat(dfs, ignore_index=True)
len(job_url_data_df)

60981

In [6]:
job_url_data_df = job_url_data_df.drop_duplicates()
len(job_url_data_df)

21260

In [7]:
job_url_data_df.to_json("/Users/annettedblackburn/Desktop/CompSoc/Take_Home_1/job_url_dataframe.json")
job_url_data_df.head()

,job_url
0,https://www.indeed.com/rc/clk?jk=64568c71be4aa...
1,https://www.indeed.com/rc/clk?jk=7b12bce39025f...
2,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
3,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
4,https://www.indeed.com/rc/clk?jk=16d3c0f3ed372...


In [8]:
# Names of Files

#("/Users/annettedblackburn/Desktop/CompSoc/Take_Home_1/indeed_scraped_data/job_url_data/job_urls_for_parsehub_5172022_v2.csv")
#("/Users/annettedblackburn/Desktop/CompSoc/Take_Home_1/indeed_scraped_data/job_url_data/job_urls_for_parsehub_5172022_v3.csv")
#("/Users/annettedblackburn/Desktop/CompSoc/Take_Home_1/indeed_scraped_data/job_url_data/job_urls_for_parsehub_5172022_v4.csv")
#("/Users/annettedblackburn/Desktop/CompSoc/Take_Home_1/indeed_scraped_data/job_url_data/job_urls_for_parsehub_5172022_v5.csv")

#("/Users/annettedblackburn/Desktop/CompSoc/Take_Home_1/indeed_scraped_data/job_url_data/job_urls_for_parsehub_5182022_v1.csv")

#("/Users/annettedblackburn/Desktop/CompSoc/Take_Home_1/indeed_scraped_data/job_url_data/job_urls_for_parsehub_5192022_v1.csv")
#("/Users/annettedblackburn/Desktop/CompSoc/Take_Home_1/indeed_scraped_data/job_url_data/job_urls_for_parsehub_5192022_v2.csv")

#("/Users/annettedblackburn/Desktop/CompSoc/Take_Home_1/indeed_scraped_data/job_url_data/job_urls_for_parsehub_5202022_v1.csv")
#("/Users/annettedblackburn/Desktop/CompSoc/Take_Home_1/indeed_scraped_data/job_url_data/job_urls_for_parsehub_5202022_v2.csv")

#("/Users/annettedblackburn/Desktop/CompSoc/Take_Home_1/indeed_scraped_data/job_url_data/job_urls_for_parsehub_5212022_v1.csv")
#("/Users/annettedblackburn/Desktop/CompSoc/Take_Home_1/indeed_scraped_data/job_url_data/job_urls_for_parsehub_5212022_v2.csv")

#("/Users/annettedblackburn/Desktop/CompSoc/Take_Home_1/indeed_scraped_data/job_url_data/job_urls_for_parsehub_5222022_v1.csv")
#("/Users/annettedblackburn/Desktop/CompSoc/Take_Home_1/indeed_scraped_data/job_url_data/job_urls_for_parsehub_5222022_v2.csv")

#("/Users/annettedblackburn/Desktop/CompSoc/Take_Home_1/indeed_scraped_data/job_url_data/job_urls_for_parsehub_5232022_v1.csv")
#("/Users/annettedblackburn/Desktop/CompSoc/Take_Home_1/indeed_scraped_data/job_url_data/job_urls_for_parsehub_5232022_v2.csv")

In [ ]:
# adding dates to each CSV
# job_url_data_df.insert(0, "Date", [5-17-22, 5-18-22, 5-19-22, 5-20-22, 5-21-22, 5-22-22, 5-23-22], True)
# I'm not sure how to insert the dates for each URL in each CSV for each day, rather than simply adding the dates to only 7 URLs
# I'm certain that I need this to be able to analyze the missing data distribution by date for Task 3

#### Task 2: Clean and integrate information from the “job_info_data” folder into one data frame. Files from this subfolder might have two different formats. Some of them are csv files, while others are json files. The columns might also be named differently. Find ways to read each of the files into pandas, drop records with missing job titles and/or missing job descriptions, and combine them into one dataframe. Lastly, drop records with duplicate job urls, and then save them into a separate csv file. How many unique jobs are there in the cleaned dataframe? 

#### There are __15,784__ unique jobs in the cleaned dataframe.

In [9]:
def format_json_df(df):
    elements = []
    for idx, row in df.iterrows():
        elements += [row['lnks']['link']]
    # drop duplicates
    df = pd.DataFrame({'lnks_link': elements})
    df = df.drop_duplicates()
    return df

def format_csv_df(df):
    # drop duplicates
    df = df.drop_duplicates()
    # drop NaNs
    df = df.dropna()
    return df

In [10]:
file_root = "/Users/annettedblackburn/Desktop/CompSoc/Take_Home_1/indeed_scraped_data/job_info_data"
dfs = []
json_dfs = []
csv_dfs = []
for file in os.listdir(file_root):
    if file == ".DS_Store": continue
    file_path = f"{file_root}/{file}"

    file_extension = file[-3:]
    # Indexing to detemrine if file is JSON or CSV
    if file_extension == "csv":
        df = pd.read_csv(file_path)
        df = format_csv_df(df)
        csv_dfs += [df]
    else:
        df = pd.read_json(file_path)#, lines=True, encoding="utf8")
        df = format_json_df(df)
        json_dfs += [df]
    
    dfs += [df]

# join all DFs together
job_info_data_df = pd.concat(dfs, ignore_index=True)
len(job_info_data_df)

# drop duplicates, drop records with missing job titles and/or missing job descriptions, & drop duplicate URLs
job_info_data_df = format_csv_df(job_info_data_df)

job_info_data_df.head()

job_info_data_df.to_csv("/Users/annettedblackburn/Desktop/CompSoc/Take_Home_1/job_info.csv")


In [11]:
csv = csv_dfs[0]
csv.head()

,lnks_link,lnks_job_title,lnks_company,lnks_company_url,lnks_company_location,lnks_job_description
559,https://www.indeed.com/rc/clk?jk=069416ccc58dc...,Maintenance Controller (A&P) Technician,"Total Aviation Staffing, LLC",https://www.indeed.com/cmp/Total-Aviation-Staf...,"Washington, DC 20500",Do you want to be part of a world-class team w...
560,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...,Program Manager,Norton Staffing and Recruiting Group,https://www.indeed.com/cmp/Norton-Staffing-and...,"Meriden, CT",BH: 3.1\n***Our Client is looking for a Progra...
562,https://www.indeed.com/rc/clk?jk=263b35a70c9a8...,"Senior Hydrogeologist, Boulder, Colorado",Formation Environmental,https://www.indeed.com/cmp/Formation-Environme...,"Boulder, CO 80301",Senior Hydrogeologist\nFormation Environmental...
563,https://www.indeed.com/rc/clk?jk=e15dbd7ec34e6...,Contract Administration Mgr I,Dept of Family & Protectve Svc,https://www.indeed.com/cmp/Texas-Health-and-Hu...,"Austin, TX 78751",Contract Administration Mgr I - THIS POISTION ...
564,https://www.indeed.com/rc/clk?jk=d0e4f6dc1721c...,Sr. Quality Assurance,Amgen,https://www.indeed.com/cmp/Amgen?campaignid=mo...,"Cambridge, MA 02139",HOW MIGHT YOU DEFY IMAGINATION?\nYou’ve earned...


In [12]:
json = json_dfs[0]
json.head()

,lnks_link
0,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
1,https://www.indeed.com/rc/clk?jk=a95cd2409de80...
2,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
3,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
4,https://www.indeed.com/rc/clk?jk=df119ccd5529f...


In [13]:
json.iat[0, 0]

'https://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0DPiPBVS3KmjTqjymWZvPzPJEsEe9JFHhSvGO4UmOZP0kOy32CS6_PT7hbGwRraUefevT5uKfDJ58DCrs1u2n20-acAq4xHlznVkm-Os4t7F1qSSqUfbeARoCETXdHx3W8oG0cxzBfvqNNbfFJYFsMYy2t8AAyq4hnYB1GCJvQnZ-5sv0JufMAs200nJqN_DUzm4b8XynUuVB532UNHUkve_CFwfb4A1iTdWvc9iDCX9w321LGUK2R8POoXCdI2BKAyYZVVyYSwm8TrInNLwMGbysppDdYjLhmoRpl9dA7O_pbVeobqtCa_9OaxbRdaREfHBVRI9cwIYEFUSjOC1T9xrqcormXhVgWcbHewzXI4D5DS6TdFP5Ck7frQTev1zHeaUiHXnR3BipOuL_zpxHwIIIKqGqcsgtKsBxR2nZDLqglnHmvAy0uWtGD5a6rhjDn3SjjVE7JBkYRg_FbzvLGLoulimLSEJqmg36LeCNhy1F4qAhgxeUOF71ipbbVJPCMNO1i1Bc-Vl2ZeW-7CCzDoo52k21XIlOVs-br4UB17ZwuyEf50vRaBdLV0X7s1G-Fyzu859tUfzhHMsV4TN4JjS4X__EyTt8W6NSzR1QkKaL6W83rfPwOydE7ocPyVvwk7mDMs8coMMPjRhtRsd7SFRMWZiH-kAVaoGLVAMK_RSvm3tux3D-Z-pF7FmyDn1qHRLNpUPHD_DL0mNxyAMigzk8s1KTm88sB23Gl8DaAGweqfS9t3hNT14DKRj-wwzIz2QvpLSK1WoOWL-z3yWey7Q-60Iw_WTueukUWA4VMrvSggF3GlJkKzgTnVy1N_OyBdIyDmXyw0f2KoDLbR6kHJ&p=2&fvj=0&vjs=3'

In [14]:
len(job_info_data_df)


15784

#### Task 3: Merge between “job_url_data” and “job_info_data”. What is the percentage of jobs that can be matched between these two data sources? How are the missing data (unmatched job urls) distributed by date? What about matched job urls? How many complete job listings were we able to collect each day? How would you interpret this result with respect to data quality? Does it mean that our data collection strategy is flawed and thus introduces non-random sampling biases? 

#### The percentage of jobs that can matched between the two data sources is roughly __79.7%__.
#### The missing data (unmatched job URLs) are distributed by date by ________.
#### The matched job URLs are distributed by date by _____.
#### There are ____ complete job listings collected each day.
#### This means _____ with respect to the quality of the data.
#### __Yes__, the data collection strategy is flawed because 20% of the data are missing and it's acceptable to have 5% or less missing data.

In [ ]:
job_info_data_df['job_url'] = job_info_data_df.pop('lnks_link')

In [21]:
job_info_data_df.head()

,lnks_job_title,lnks_company,lnks_company_url,lnks_company_location,lnks_job_description,job_url
0,Maintenance Controller (A&P) Technician,"Total Aviation Staffing, LLC",https://www.indeed.com/cmp/Total-Aviation-Staf...,"Washington, DC 20500",Do you want to be part of a world-class team w...,https://www.indeed.com/rc/clk?jk=069416ccc58dc...
1,Program Manager,Norton Staffing and Recruiting Group,https://www.indeed.com/cmp/Norton-Staffing-and...,"Meriden, CT",BH: 3.1\n***Our Client is looking for a Progra...,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
2,"Senior Hydrogeologist, Boulder, Colorado",Formation Environmental,https://www.indeed.com/cmp/Formation-Environme...,"Boulder, CO 80301",Senior Hydrogeologist\nFormation Environmental...,https://www.indeed.com/rc/clk?jk=263b35a70c9a8...
3,Contract Administration Mgr I,Dept of Family & Protectve Svc,https://www.indeed.com/cmp/Texas-Health-and-Hu...,"Austin, TX 78751",Contract Administration Mgr I - THIS POISTION ...,https://www.indeed.com/rc/clk?jk=e15dbd7ec34e6...
4,Sr. Quality Assurance,Amgen,https://www.indeed.com/cmp/Amgen?campaignid=mo...,"Cambridge, MA 02139",HOW MIGHT YOU DEFY IMAGINATION?\nYou’ve earned...,https://www.indeed.com/rc/clk?jk=d0e4f6dc1721c...


In [27]:
# merge 
job_url_info_merged_df = job_info_data_df.merge(job_url_data_df, on=['job_url'])

In [34]:
unique_url_count = len(pd.unique(job_url_info_merged_df['job_url']))
print("Number of unique URLs:", unique_url_count)

Number of unique URLs: 12592


In [35]:
# getting percentage of data that are matched between two sources from unique merged values and job listing URLs from job_info_data_df
12592/15784


0.7977698935631019